# autoreload notebook magic

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

# Imports

In [3]:
'''
Train respeller model

We backpropagate loss from pretrained TTS model to a Grapheme-to-Grapheme (G2G) respeller model to help it respell words
into a simpler form

Intermediated respellings are discrete character sequences
We can backpropagate through these using gumbel softmax and the straight through estimator
'''
import argparse
import numpy as np
import torch
import torch.distributed as dist
import torch.nn.functional as F
from torch.utils.data import DataLoader
import json
import glob
import re
from collections import defaultdict

from fastpitch import models as fastpitch_model
from fastpitch.common.text.text_processing import TextProcessor

from modules.model import EncoderRespeller
from modules.gumbel_vector_quantizer import GumbelVectorQuantizer
from modules.sdtw_cuda_loss import SoftDTW

import librosa
import librosa.display
import matplotlib
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image

import fastpitch.common.tb_dllogger as logger
from torch_optimizer import Lamb
import time
from fastpitch.common.utils import mask_from_lens
from collections import OrderedDict

import os
from collections import Counter
from tqdm import tqdm

In [4]:
torch.cuda.device_count()

1

# simulate running script at command line with arguments

## set experiment name

In [5]:
gamma = 0.1
lr = 0.1 # def for lamb optimizer is 0.001
dist_metric = 'l1'

wandb_project_name = "respeller-training"
exp_name = f"test_development"
# fastpitch_chkpt = 'fastpitch/exps/halved_ljspeech_data_nospaces_noeos/FastPitch_checkpoint_1000.pt' # 'fastpitch/exps/halved_ljspeech_data/FastPitch_checkpoint_1000.pt',
fastpitch_chkpt = 'fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt'

## CLAs that affect training loop in this notebook!!!

In [6]:
# imitate CLAs
import sys
sys.argv = [
    'train.py',
    '--wandb-project-name', wandb_project_name,
    '--chkpt-save-dir', f'/home/s1785140/respeller/exps/{exp_name}', 
    '--fastpitch-chkpt', fastpitch_chkpt,
    '--input-type', 'char',
    '--symbol-set', 'english_pad_lowercase_nopunc',
    # '--text-cleaners', 'lowercase_no_punc',
    '--use-mas',
    '--cuda',
    '--n-speakers', '1',
    '--use-sepconv',
    # '--add-spaces',
    # '--eos-symbol', '$',
    '--batch-size', '4',
    '--val-num-to-gen', '2',
    '--softdtw-temp', str(gamma),
    '--dist-func', dist_metric,
    '--learning-rate', str(lr),
    
    # NB for real training!
    # '--epochs', '10000', 
    # '--val-log-interval', '10',
    # '--resume', # resume from latest checkpoint
    
    # # NB for development!
    '--epochs', '2', # NB for development!
    '--val-log-interval', '1', # NB for development!
    '--max-iters-per-epoch', '5', # NB for development!
    # '--skip-before-train-loop-validation', # NB for development!
    # '--freeze-embedding-table',
]

In [7]:
# print the command to be run so we can copy and paste it into terminal if needs be
print(f"python", " ".join(sys.argv))

python train.py --wandb-project-name respeller-training --chkpt-save-dir /home/s1785140/respeller/exps/test_development --fastpitch-chkpt fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt --input-type char --symbol-set english_pad_lowercase_nopunc --use-mas --cuda --n-speakers 1 --use-sepconv --batch-size 4 --val-num-to-gen 2 --softdtw-temp 0.1 --dist-func l1 --learning-rate 0.1 --epochs 2 --val-log-interval 1 --max-iters-per-epoch 5


In [8]:
def prettyprint_cla_for_training(sys_argv):
    """for every flag and its arguments print on a new line"""
    def print_line(to_print):
        s = "    " + " ".join(to_print)
        print(s)

    if sys_argv[0].endswith('.py'):
        args = sys_argv[1:]
    else: 
        args = sys_argv
        
    to_print = []        
    for i, arg in enumerate(args):            
        if arg.startswith('--') and i != 0:
            print_line(to_print + ['\\'])
            to_print = []
        to_print.append(arg)
        
    print_line(to_print)
            
print("cd /home/s1785140/respeller")
print("python train.py \\")
prettyprint_cla_for_training(sys.argv)

cd /home/s1785140/respeller
python train.py \
    --wandb-project-name respeller-training \
    --chkpt-save-dir /home/s1785140/respeller/exps/test_development \
    --fastpitch-chkpt fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt \
    --input-type char \
    --symbol-set english_pad_lowercase_nopunc \
    --use-mas \
    --cuda \
    --n-speakers 1 \
    --use-sepconv \
    --batch-size 4 \
    --val-num-to-gen 2 \
    --softdtw-temp 0.1 \
    --dist-func l1 \
    --learning-rate 0.1 \
    --epochs 2 \
    --val-log-interval 1 \
    --max-iters-per-epoch 5


# print sbatch.sh hyperparam sweep training commands

In [17]:
def convert_to_str_for_path(obj):
    if type(obj) == tuple or list:
        return ",".join([str(x) for x in obj])
    return str(obj)

def get_highlighted_param_values(hparams, sweep_params_dict, keys_to_add_to_descriptor):
    param_and_values = []
    for key in keys_to_add_to_descriptor:
        if key in sweep_params_dict: # search this dict first as we prefer to used individual param values
            v = sweep_params_dict[key]
        elif key in hparams[key]:
            v = hparams[key]
        else:
            raise ValueError("key {key} not found")
        v = convert_to_str_for_path(v)
        param_and_values.append((key,v))
        
    return param_and_values

In [20]:
import itertools

def print_train_commands_for_hparam_sweep(exp_name, hparams, prettyprint):
    # TODO add prettyprint option if false  command is all on one line, 
    # might make pasting a lot of jobs more robust through terminal
    
    product = list(itertools.product(*[
        hparams['gammas'],
        hparams['lrs'],
        hparams['arch_cfgs'],
        hparams['freeze_embeddings'],
        hparams['embedding_dim'],
        hparams['seeds'],
        hparams['src_key_padding_mask'],
        hparams['gumbel_temps'],
        hparams['desired_text_lens'],
        hparams['dropout_layers'],
        hparams['concat_pos_encoding'],
    ]))
    
    for i, (
        gamma, 
        lr, 
        arch, 
        freeze, 
        embed_dim, 
        seed, 
        padmask,
        gumbel_temp,
        desired_text_len,
        dropout_layer,
        concat_pos_encoding,
    ) in enumerate(product):
        # create additional run descriptor        
        sweep_params_dict = {
            'gamma': gamma,'lr': lr,'freeze': freeze,'embed_dim': embed_dim,'seed': seed,
            'padmask': padmask,'gumbel_temp': gumbel_temp,'desired_text_len': desired_text_len,
            'dropout_layer': dropout_layer, 'concat_pos_encoding': concat_pos_encoding,
        }
        param_and_values = get_highlighted_param_values(hparams, sweep_params_dict, hparams['keys_to_add_to_descriptor'])
        highlighted_model_params = "::".join([f"{p}={v}" for p, v in param_and_values])
        model_dir = f"{exp_name}::{highlighted_model_params}"
        
        sys_argv = [
            'train.py',
            '--wandb-project-name', hparams['wandb_project_name'],
            '--chkpt-save-dir', f'/home/s1785140/respeller/exps/{model_dir}', 
            '--fastpitch-chkpt', hparams['tts_model_path'],
            '--input-type', 'char',
            # '--symbol-set', 'english_basic_lowercase_no_arpabet',
            '--symbol-set', 'english_pad_lowercase_nopunc',
            '--text-cleaners', 'lowercase_no_punc',
            '--use-mas',
            '--cuda',
            '--n-speakers', '1',
            '--use-sepconv',
            '' if padmask else '--no-src-key-padding-mask',
            '--respelling-len-modifier', str(desired_text_len),
            '--nheads', str(arch['nheads']),
            '--num-layers', str(arch['num_layers']),
            '--d-model', str(arch['d_model']),
            '--d-feedforward', str(arch['ffdim']),
            '--dropout-inputs', str(arch['dropout_inputs']),
            '--dropout-layers', str(dropout_layer),
            '--embedding-dim', str(embed_dim),
            '--freeze-embedding-table' if freeze else '',
            '--pretrained-embedding-table' if hparams['pretrained_embedding_table'] else '',
            '--concat-pos-encoding' if concat_pos_encoding else '',
            '--gumbel-temp', str(gumbel_temp[0]), str(gumbel_temp[1]), str(gumbel_temp[2]),
            '--batch-size', str(hparams['bsz']),
            '--seed', str(seed),
            '--val-num-to-gen', '32',
            '--softdtw-temp', str(gamma),
            '--dist-func', "l1",
            '--learning-rate', str(lr),
            '--epochs', str(hparams['epochs']), 
            '--val-log-interval', '20',
            '--epochs-per-checkpoint', '20',
            '--resume' if hparams['resume'] else '',
        ]
        print()
        print(f"# Experiment {i+1}/{len(product)}: {model_dir}")
        print("./sbatch.sh python train.py \\")
        prettyprint_cla_for_training(sys_argv)
    print("\n")

print("cd /home/s1785140/respeller")

exps = {
    # 18th jan 2023 sweep gammas and freeze embeddings
    'SweepGammaFreeze': {
        'bsz': 32,
        'wandb_project_name': 'respeller',
        'tts_model_path': fastpitch_chkpt,
        'gammas': [10.0,1.0,0.1,0.01], 
        'lrs': [0.1],
        'arch_cfgs': [
            # {'nheads': 4, 'num_layers': 4, 'd_model':512},
            # {'nheads': 2, 'num_layers': 2, 'd_model':256},
            {'nheads': 1, 'num_layers': 1, 'd_model':128},
        ],
        'embedding_dim': [384],
        'freeze_embeddings': [True, False],
        'pretrained_embedding_table': True,
        'seeds': [1337],
        'resume': False,
    },
    
    # 18th jan 2023 init embedding table from scratch and change embedding table size
    'RandInitEmbedding': {
        'bsz': 32,
        'wandb_project_name': 'respeller',
        'tts_model_path': fastpitch_chkpt,
        'gammas': [1.0], 
        'lrs': [0.1],
        'arch_cfgs': [
            {'nheads': 1, 'num_layers': 1, 'd_model':128},
        ],
        'embedding_dim': [384, 128, 64],
        'freeze_embeddings': [False],
        'pretrained_embedding_table': False,
        'seeds': [1337],
        'resume': False,
    },
    
    # random seed exp to investigate collapse of b to p in respellings
    '10RandSeeds': {
        'bsz': 32,
        'wandb_project_name': 'respeller',
        'tts_model_path': fastpitch_chkpt,
        'gammas': [1.0], 
        'lrs': [0.1],
        'arch_cfgs': [
            {'nheads': 1, 'num_layers': 1, 'd_model':128},
        ],
        'embedding_dim': [384],
        'freeze_embeddings': [True],
        'pretrained_embedding_table': True,
        'seeds': [1337, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        'resume': False,
    },
    
    # random seed exp to investigate collapse of b to p in respellings
    'Sweep_PaddingMask_DropoutInputs_FFdim': {
        'bsz': 128,
        'wandb_project_name': 'respeller',
        'tts_model_path': fastpitch_chkpt,
        'gammas': [1.0], 
        'lrs': [0.1],
        'arch_cfgs': [
            {'nheads': 2, 'num_layers': 1, 'd_model':128, 'dropout_inputs': 0.0, 'ffdim': 2048},
            {'nheads': 2, 'num_layers': 1, 'd_model':128, 'dropout_inputs': 0.1, 'ffdim': 2048},
            {'nheads': 2, 'num_layers': 1, 'd_model':128, 'dropout_inputs': 0.0, 'ffdim': 512},
            {'nheads': 2, 'num_layers': 1, 'd_model':128, 'dropout_inputs': 0.1, 'ffdim': 512},
        ],
        'embedding_dim': [384],
        'freeze_embeddings': [True],
        'pretrained_embedding_table': True,
        'seeds': [1337],
        'resume': False,
        'src_key_padding_mask': [True, False],
        'gumbel_temp': [(2, 0.5, 0.999995)],
    },
    
    'Sweep_DesiredTextLen': {
        'keys_to_add_to_descriptor': ['desired_text_len'],
        'bsz': 128,
        'epochs': 2500,
        'wandb_project_name': 'respeller',
        'tts_model_path': 'fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt',
        'gammas': [1.0], 
        'lrs': [0.1],
        'arch_cfgs': [
            {'nheads': 2, 'num_layers': 1, 'd_model':128, 'dropout_inputs': 0.0, 'ffdim': 512},
        ],
        'embedding_dim': [384],
        'freeze_embeddings': [True],
        'pretrained_embedding_table': True,
        'seeds': [1337],
        'resume': False,
        'src_key_padding_mask': [True],
        'gumbel_temps': [
            (2, 0.5, 0.999995),
        ],
        'desired_text_lens': [-2,-1,0,1,2],
        'dropout_layers': [0.0],
    },
    
    'Sweep_DropoutLayer': {
        'bsz': 128,
        'epochs': 1000,
        'wandb_project_name': 'respeller',
        'tts_model_path': fastpitch_chkpt,
        'gammas': [1.0], 
        'lrs': [0.1],
        'arch_cfgs': [
            {'nheads': 2, 'num_layers': 1, 'd_model':128, 'dropout_inputs': 0.0, 'ffdim': 512},
        ],
        'embedding_dim': [384],
        'freeze_embeddings': [True],
        'pretrained_embedding_table': True,
        'seeds': [1337],
        'resume': False,
        'src_key_padding_mask': [True],
        'gumbel_temps': [
            (2, 0.5, 0.999995),
        ],
        'desired_text_lens': [0],
        'dropout_layers': [0.0,0.05,0.1,0.2,0.3],
    },
    
    'NoPuncRespellerAndFastpitch': {
        'keys_to_add_to_descriptor': ['seed'],
        'bsz': 64,
        'epochs': 2000,
        'wandb_project_name': 'respeller',
        'tts_model_path': 'fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt',
        'gammas': [1.0], 
        'lrs': [0.1],
        'arch_cfgs': [
            {'nheads': 1, 'num_layers': 1, 'd_model':128, 'dropout_inputs': 0.0, 'ffdim': 512},
        ],
        'embedding_dim': [384],
        'freeze_embeddings': [True],
        'pretrained_embedding_table': True,
        'seeds': [1337,1,2,3,4],
        'resume': False,
        'src_key_padding_mask': [True],
        'gumbel_temps': [
            (2, 0.5, 0.999995),
        ],
        'desired_text_lens': [0],
        'dropout_layers': [0.0],
    },
    
    'ConcatPosEncodingAndDropoutLayer': {
        'keys_to_add_to_descriptor': ['concat_pos_encoding', 'dropout_layer', 'seed'],
        'bsz': 64,
        'epochs': 2000,
        'wandb_project_name': 'respeller',
        'tts_model_path': 'fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt',
        'gammas': [1.0], 
        'lrs': [0.1],
        'arch_cfgs': [
            {'nheads': 2, 'num_layers': 1, 'd_model':128, 'dropout_inputs': 0.0, 'ffdim': 512},
        ],
        'embedding_dim': [384],
        'freeze_embeddings': [True],
        'pretrained_embedding_table': True,
        'seeds': [1],
        'resume': False,
        'src_key_padding_mask': [True],
        'gumbel_temps': [
            (2, 0.5, 0.999995),
        ],
        'desired_text_lens': [0],
        'dropout_layers': [0.0, 0.15, 0.3],
        'concat_pos_encoding': [True, False],
    },
    
    'SweepGumbelTempFixedSchedulesAndOnlyPredictAlpha': {
        'keys_to_add_to_descriptor': ['gumbel_temp'],
        'bsz': 64,
        'epochs': 2000,
        'wandb_project_name': 'respeller',
        'tts_model_path': 'fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt',
        'gammas': [1.0], 
        'lrs': [0.1],
        'arch_cfgs': [
            {'nheads': 2, 'num_layers': 1, 'd_model':256, 'dropout_inputs': 0.0, 'ffdim': 512},
        ],
        'embedding_dim': [384],
        'freeze_embeddings': [True],
        'pretrained_embedding_table': True,
        'seeds': [1337],
        'resume': False,
        'src_key_padding_mask': [True],
        'gumbel_temps': [
            (2, 0.5, 0.999995),
            (2, 2, 1.0), # temp never decays
            (5, 5, 1.0), # temp never decays
            (1, 1, 1.0), # temp never decays
            (10, 10, 1.0), # temp never decays
            (10, 0.1, 0.999995), # temp never decays
            (20, 20, 1.0), # temp never decays
            (0.5, 0.5, 1.0), # temp never decays
            (0.1, 0.1, 1.0), # temp never decays
            (0.01, 0.01, 1.0), # temp never decays
        ],
        'desired_text_lens': [0],
        'dropout_layers': [0.0],
        'concat_pos_encoding': [False],
    },
        
}

exp_names = [
    # 'SweepGammaFreeze',
    # 'RandInitEmbedding',
    # '10RandSeeds',
    # 'Sweep_PaddingMask_DropoutInputs_FFdim',
    # 'Sweep_DesiredTextLen',
    # 'Sweep_DropoutLayer',
    # 'NoPuncRespellerAndFastpitch',
    # 'ConcatPosEncodingAndDropoutLayer',
    'SweepGumbelTempFixedSchedulesAndOnlyPredictAlpha',
]
exp_name = exp_names[0]
hparams = exps[exp_name]
print_train_commands_for_hparam_sweep(exp_name, hparams, prettyprint=True) 


cd /home/s1785140/respeller

# Experiment 1/10: SweepGumbelTempFixedSchedulesAndOnlyPredictAlpha::gumbel_temp=2,0.5,0.999995
./sbatch.sh python train.py \
    --wandb-project-name respeller \
    --chkpt-save-dir /home/s1785140/respeller/exps/SweepGumbelTempFixedSchedulesAndOnlyPredictAlpha::gumbel_temp=2,0.5,0.999995 \
    --fastpitch-chkpt fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt \
    --input-type char \
    --symbol-set english_pad_lowercase_nopunc \
    --text-cleaners lowercase_no_punc \
    --use-mas \
    --cuda \
    --n-speakers 1 \
    --use-sepconv  \
    --respelling-len-modifier 0 \
    --nheads 2 \
    --num-layers 1 \
    --d-model 256 \
    --d-feedforward 512 \
    --dropout-inputs 0.0 \
    --dropout-layers 0.0 \
    --embedding-dim 384 \
    --freeze-embedding-table \
    --pretrained-embedding-table  \
    --gumbel-temp 2 0.5 0.999995 \
    --batch-size 64 \
    --seed 1337 \
    --val-num-to-gen 32 \
    

# dev train loop + model for respeller

# Functions

## arguments to parse

In [ ]:
def parse_args(parser):
    """Parse commandline arguments"""
    parser.add_argument('--wandb-project-name', type=str, required=True,
                        help='Which wandb project to add this job to')
    parser.add_argument('-o', '--chkpt-save-dir', type=str, required=True,
                        help='Directory to save checkpoints')
    parser.add_argument('-d', '--dataset-path', type=str, default='./',
                        help='Path to dataset')
    parser.add_argument('--log-file', type=str, default=None,
                        help='Path to a DLLogger log file')

    train = parser.add_argument_group('training setup')
    train.add_argument('--cuda', action='store_true',
                      help='Enable GPU training')
    train.add_argument('--batch-size', type=int, default=16,
                      help='Batchsize (this is divided by number of GPUs if running Data Distributed Parallel Training)')
    train.add_argument('--val-num-to-gen', type=int, default=32,
                      help='Number of samples to generate in validation (determines how many samples show up in wandb')
    train.add_argument('--seed', type=int, default=1337,
                       help='Seed for PyTorch random number generators')
    train.add_argument('--grad-accumulation', type=int, default=1,
                       help='Training steps to accumulate gradients for')
    train.add_argument('--epochs', type=int, default=100, #required=True,
                       help='Number of total epochs to run')
    train.add_argument('--max-iters-per-epoch', type=int, default=None, 
                       help='Number of total batches to iterate through each epoch (reduce this to small number to quickly test whole training loop)')
    train.add_argument('--epochs-per-checkpoint', type=int, default=10,
                       help='Number of epochs per checkpoint')
    train.add_argument('--checkpoint-path', type=str, default=None,
                       help='Checkpoint path to resume train')
    train.add_argument('--resume', action='store_true',
                       help='Resume train from the last available checkpoint')
    train.add_argument('--val-log-interval', type=int, default=5,
                       help='How often to generate melspecs/audio for respellings and log to wandb')
    train.add_argument('--speech-length-penalty-training', action='store_true',
                       help='Whether or not to encourage model to output similar length outputs\
                       as the ground truth. Idea from V2C: Visual Voice Cloning (Chen et al. 2021)')
    train.add_argument('--skip-before-train-loop-validation', action='store_true',
                       help='Skip running validation before model training begins (mostly for speeding up testing of actual training loop)')
    train.add_argument('--avg-loss-by-speech_lens', action='store_true',
                       help='Average the softdtw loss according to number of timesteps in predicted sequence')
    train.add_argument('--softdtw-temp', type=float, default=0.01,
                      help='How hard/soft to make min operation. Minimum is recovered by setting this to 0.')
    train.add_argument('--softdtw-bandwidth', type=int, default=120,
                      help='Bandwidth for pruning paths in alignment matrix when calculating SoftDTW')
    train.add_argument('--dist-func', type=str, default="l1",
                       help='What distance function to use in softdtw loss calculation')
    
    opt = parser.add_argument_group('optimization setup')
    opt.add_argument('--optimizer', type=str, default='lamb', choices=['adam', 'lamb'],
                     help='Optimization algorithm')
    opt.add_argument('-lr', '--learning-rate', default=0.1, type=float,
                     help='Learning rate')
    opt.add_argument('--weight-decay', default=1e-6, type=float,
                     help='Weight decay')
    opt.add_argument('--grad-clip-thresh', default=1000.0, type=float,
                     help='Clip threshold for gradients')
    opt.add_argument('--warmup-steps', type=int, default=1000,
                     help='Number of steps for lr warmup')

    arch = parser.add_argument_group('architecture')
    arch.add_argument('--d-model', type=int, default=512,
                       help='Hidden dimension of tranformer')
    arch.add_argument('--latent-temp', type=tuple, default=(2, 0.5, 0.999995),
                       help='Temperature annealling parameters for Gumbel-Softmax (start, end, decay)')
    arch.add_argument('--freeze-embedding-table', action='store_true',
                      help='Whether or not to allow grapheme embedding input table for EncoderRespeller to be updated.')
    
    parser.add_argument('--feature', action='store_true')

    pretrained_tts = parser.add_argument_group('pretrained tts model')
    # pretrained_tts.add_argument('--fastpitch-with-mas', type=bool, default=True,
    #                   help='Whether or not fastpitch was trained with Monotonic Alignment Search (MAS)')
    pretrained_tts.add_argument('--fastpitch-chkpt', type=str, required=True,
                      help='Path to pretrained fastpitch checkpoint')
    pretrained_tts.add_argument('--input-type', type=str, default='char',
                      choices=['char', 'phone', 'pf', 'unit'],
                      help='Input symbols used, either char (text), phone, pf '
                      '(phonological feature vectors) or unit (quantized acoustic '
                      'representation IDs)')
    pretrained_tts.add_argument('--symbol-set', type=str, default='english_basic_lowercase',
                      help='Define symbol set for input sequences. For quantized '
                      'unit inputs, pass the size of the vocabulary.')
    pretrained_tts.add_argument('--n-speakers', type=int, default=1,
                      help='Condition on speaker, value > 1 enables trainable '
                      'speaker embeddings.')
    # pretrained_tts.add_argument('--use-sepconv', type=bool, default=True,
    #                   help='Use depthwise separable convolutions')
    
    audio = parser.add_argument_group('log generated audio')
    audio.add_argument('--hifigan', type=str, default='/home/s1785140/pretrained_models/hifigan/ljspeech/LJ_V1/generator_v1',
                       help='Path to HiFi-GAN audio checkpoint')
    audio.add_argument('--hifigan-config', type=str, default='/home/s1785140/pretrained_models/hifigan/ljspeech/LJ_V1/config.json',
                       help='Path to HiFi-GAN audio config file')
    audio.add_argument('--sampling-rate', type=int, default=22050,
                       help='Sampling rate for output audio')
    audio.add_argument('--hop-length', type=int, default=256,
                       help='STFT hop length for estimating audio length from mel size')
    
    data = parser.add_argument_group('dataset parameters')
    data.add_argument('--wordaligned-speechreps', type=str, default='/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels',
                       help='Path to directory of wordaligned speechreps/mels. Inside are folders\
                       each named as a wordtype and containing tensors of word aligned speechreps for each example')
    data.add_argument('--train-wordlist', type=str, default='/home/s1785140/data/ljspeech_fastpitch/respeller_train_words.json',
                       help='Path to words that are used to train respeller')
    data.add_argument('--val-wordlist', type=str, default='/home/s1785140/data/ljspeech_fastpitch/respeller_dev_words.json',
                       help='Path to words that are used to report validation metrics for respeller')
    data.add_argument('--max-examples-per-wordtype', type=int, default=1,
                       help='Path to words that are used to report validation metrics for respeller')
    # data.add_argument('--add-spaces', action='store_true',
    #                   help='Whether to add leading space and trailing EOS symbol to word during training and test.')
    
    
    cond = parser.add_argument_group('conditioning on additional attributes')
    dist = parser.add_argument_group('distributed training setup')

    return parser

def load_checkpoint(args, model, filepath):
    if args.local_rank == 0:
        print(f'Loading model and optimizer state from {filepath}')
    checkpoint = torch.load(filepath, map_location='cpu')
    sd = {k.replace('module.', ''): v
          for k, v in checkpoint['state_dict'].items()}
    getattr(model, 'module', model).load_state_dict(sd)
    return model

def load_respeller_checkpoint(args, model, filepath, optimizer, epoch, total_iter):
    if args.local_rank == 0:
        print(f'Loading model and optimizer state from {filepath}')
    checkpoint = torch.load(filepath, map_location='cpu')
    epoch[0] = checkpoint['epoch'] + 1
    total_iter[0] = checkpoint['iteration']
    sd = {k.replace('module.', ''): v
          for k, v in checkpoint['state_dict'].items()}
    getattr(model, 'module', model).load_state_dict(sd)
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model

def last_checkpoint(output):
    saved = sorted(
        glob.glob(f'{output}/respeller_checkpoint_*.pt'),
        key=lambda f: int(re.search('_(\d+).pt', f).group(1)))

    def corrupted(fpath):
        try:
            torch.load(fpath, map_location='cpu')
            return False
        except:
            warnings.warn(f'Cannot load {fpath}')
            return True

    if len(saved) >= 1 and not corrupted(saved[-1]):
        return saved[-1]
    elif len(saved) >= 2:
        return saved[-2]
    else:
        return None

def init_embedding_weights(source_tensor, target_tensor):
    """copy weights inplace from source tensor to target tensor"""
    target_tensor.requires_grad = False
    target_tensor.copy_(source_tensor.clone().detach())
    target_tensor.requires_grad = True

def load_pretrained_fastpitch(args):
    # load chkpt
    device = torch.device('cuda' if args.cuda else 'cpu')
    model_config = fastpitch_model.get_model_config('FastPitch', args)
    fastpitch = fastpitch_model.get_model('FastPitch', model_config, device, forward_is_infer=True)
    load_checkpoint(args, fastpitch, args.fastpitch_chkpt)
    # get information about grapheme embedding table
    n_symbols = fastpitch.encoder.word_emb.weight.size(0)
    grapheme_embedding_dim = fastpitch.encoder.word_emb.weight.size(1)
    return fastpitch, n_symbols, grapheme_embedding_dim, model_config

# beginning of main(), parse Command Line Args

In [38]:
parser = argparse.ArgumentParser(description='PyTorch Respeller Training', allow_abbrev=False)
parser = parse_args(parser)
args, _unk_args = parser.parse_known_args()

parser = fastpitch_model.parse_model_args('FastPitch', parser)
args, unk_args = parser.parse_known_args()
if len(unk_args) > 0:
    raise ValueError(f'Invalid options {unk_args}')

if args.cuda:
    args.num_gpus = torch.cuda.device_count()
    args.distributed_run = args.num_gpus > 1
    args.batch_size = int(args.batch_size / args.num_gpus)
else:
    args.distributed_run = False

torch.manual_seed(args.seed)
np.random.seed(args.seed)

if args.distributed_run:
    mp.spawn(train, nprocs=args.num_gpus, args=(args,))

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'


# WANDB - weights and biases init

In [39]:
import wandb
wandb.login() # needed for wandb integration with jupyter notebook

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [40]:
%env "WANDB_NOTEBOOK_NAME" "respeller-dev-train-ipynb"

env: "WANDB_NOTEBOOK_NAME"="respeller-dev-train-ipynb"


## set project name and run name

In [41]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y-%H:%M:%S")
print("date and time =", dt_string)

now = 2023-01-24 15:07:06.916894
date and time = 24/01/2023-15:07:06


In [42]:
run = 1

project_name = args.wandb_project_name #"respeller-dev-train-ipynb"
exp_name = "test_dev"
experiment_name = f"{exp_name}_run-{run}_{dt_string}"

# store important information into WANDB config for easier tracking of experiments
# add all key values from parser
wandb_config = vars(args)
wandb.init(
    project=project_name,
    name=experiment_name,
    config=wandb_config,
)

Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/site-packages/wandb/sdk/internal/system/system_monitor.py", line 118, in _start
    asset.start()
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/cpu.py", line 166, in start
    self.metrics_monitor.start()
  File "/home/s1785140/miniconda3/envs/fastpitch/lib/python3.8/site-packages/wandb/sdk/internal/system/assets/interfaces.py", line 168, in start
    logger.info(f"Started {self._process.name}")
AttributeError: 'NoneType' object has no attribute 'name'
Exception in thread SystemMonitor:
Traceback (most recent call last):
  File "/hom

# 'train()' - forward pass through model to get loss

## create / load models 

In [43]:
rank = 0
device = 'cuda'

args.local_rank = rank
tts, n_symbols, grapheme_embedding_dim, model_config = load_pretrained_fastpitch(args)
tts.to(device)

respeller = EncoderRespeller(n_symbols=n_symbols, pretrained_tts=tts, d_model=args.d_model)
respeller.to(device)

# quantiser = GumbelVectorQuantizer(
#     in_dim=args.d_model,
#     codebook_size=n_symbols,  # number of codebook entries
#     embedding_dim=grapheme_embedding_dim,
#     temp=args.latent_temp,
# )
# quantiser.to(device)

# init_embedding_weights(tts.encoder.word_emb.weight.unsqueeze(0), quantiser.vars)


# batch_size, len_x, len_y, dims = 8, 15, 12, 5
# x = torch.rand((batch_size, len_x, dims), requires_grad=True)
# y = torch.rand((batch_size, len_y, dims))

# criterion = SoftDTW(use_cuda=True, gamma=0.1, dist_func=F.mse_loss)
criterion = SoftDTW(use_cuda=True, gamma=args.softdtw_temp)
# input should be size [bsz, seqlen, dim]
criterion.to(device)

Loading model and optimizer state from fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt


SoftDTW()

### load HiFiGAN vocoder 

In [44]:
def load_vocoder(args, device):
    """Load HiFi-GAN vocoder from checkpoint"""
    checkpoint_data = torch.load(args.hifigan)
    vocoder_config = fastpitch_model.get_model_config('HiFi-GAN', args)
    vocoder = fastpitch_model.get_model('HiFi-GAN', vocoder_config, device)
    vocoder.load_state_dict(checkpoint_data['generator'])
    vocoder.remove_weight_norm()
    vocoder.eval()
    return vocoder

In [45]:
vocoder = load_vocoder(args, device)

## forward pass through model with dummy data

In [46]:
# batches = []
# symbol_set = 'english_basic_lowercase'
# text_cleaners = []
# gt_log_mel = torch.load('/home/s1785140/data/ljspeech_fastpitch/mels/LJ001-0001.pt').cuda().unsqueeze(0).transpose(1,2) # intro batch dimension + [bsz, seqlen, dim]
# raw_text = 'printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the exhibition'

# # process text using same processor as fastpitch
# tp = TextProcessor(symbol_set, text_cleaners)
# text = torch.LongTensor(tp.encode_text(raw_text)).unsqueeze(0).cuda()

# batches.append((text, gt_log_mel))

# # for batch in batches:
# batch = batches[0]
    
# ###############################################################################################################
# # text, ssl_reps, e2e_asr_predictions, gt_log_mel = batch
# text, gt_log_mel = batch

# ###############################################################################################################
# # create inputs
# # if args.use_acoustic_input:
# #     inputs = inputs.concat(ssl_reps)

# ###############################################################################################################
# # forward pass
# g_embeddings, g_embedding_indices = respeller(text[:13])

# print('examine the padding embedding')
# pad_idx = 0
# respeller.quantiser.vars[:,pad_idx,:] 

# masked_g_embedding_indices = torch.zeros(g_embedding_indices.size())

# masked_g_embedding_indices.size()

# g_embeddings.size()

# masked_g_embeddings = torch.zeros(g_embeddings.size())

# text_lens = [10]
# bsz = len(text_lens)
# for i, text_len in enumerate(text_lens):
#     #indices
#     indices = g_embedding_indices[i, :text_len]
#     masked_g_embedding_indices[i, :text_len] = indices
#     #embeddings
#     embeddings = g_embeddings[i, :text_len, :]
#     masked_g_embeddings[i, :text_len, :] = embeddings
    
    

# n_symbols

# g_embedding_indices.size()

# g_embeddings.size()

# padding_idx = 0
# mask = (g_embedding_indices != padding_idx).unsqueeze(2)
# mask.size()

# log_mel, dec_lens, _dur_pred, _pitch_pred = tts(g_embeddings, skip_embeddings=True, ids=g_embedding_indices)
# # log_mel [bsz, dim, seqlen]
# log_mel = log_mel.transpose(1,2)
# # log_mel [bsz, seqlen, dim]

# print(f'{log_mel.size()=}')
# print(f'{gt_log_mel.size()=}')

# ###############################################################################################################
# # calculate val_losses
# # respelling_loss = respelling_loss_fn(respelling, e2e_asr_predictions)
# acoustic_loss = criterion(log_mel, gt_log_mel)

# # average loss over frames 
# acoustic_loss = acoustic_loss / dec_lens
# # mel_loss = (mel_loss * mel_mask).sum() / mel_mask.sum()

# ###############################################################################################################
# # backward pass
# loss = acoustic_loss 

# print(f'{loss=}')

# # loss.backward()

# ###############################################################################################################
# # log tensorboard metrics

# ###############################################################################################################
# # validation set evaluation

# def plot_spectrogram(log_mel, figsize=(15,5), wandb_log=False, image_name=""):
#     plt.figure(figsize=figsize)
#     librosa.display.specshow(log_mel, x_axis='frames', y_axis='linear')
#     plt.colorbar()
#     if wandb_log:
#         wandb.log({image_name: wandb.Image(plt, caption=image_name)})

# batch_index = 0
# print(f'{log_mel[batch_index].transpose(0,1).size()=}')
# plot_spectrogram(log_mel[batch_index].transpose(0,1).detach().cpu().numpy())

# # # play audio
# # import IPython.display as ipd
# # audio = vocoder(log_mel[batch_index].transpose(0,1).detach().unsqueeze(0))
# # ipd.Audio(audio, rate=22050)

# batch_index = 0
# plot_spectrogram(gt_log_mel[batch_index].transpose(0,1).detach().cpu().numpy())

# develop respeller dataset

In [47]:
wordaligned_speechreps_dir = '/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels' # path to directory that contains folders of word aligned speech reps
wordlist = ['identifies','mash','player','russias','techniques'] # txt file for the words to include speech reps 

In [48]:
token_and_melfilepaths = []
for word in wordlist:
    # find all word aligned mels for the word
    word_dir = os.path.join(wordaligned_speechreps_dir, word)
    mel_files = os.listdir(word_dir)
    for mel_file in mel_files:
        mel_file_path = os.path.join(word_dir, mel_file)
        token_and_melfilepaths.append((word, mel_file_path))
    

In [49]:
token_and_melfilepaths

[('identifies',
  '/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels/identifies/identifies__LJ040-0003__occ1__seqlen68.pt'),
 ('mash',
  '/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels/mash/mash__LJ020-0011__occ1__seqlen28.pt'),
 ('player',
  '/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels/player/player__LJ041-0014__occ1__seqlen49.pt'),
 ('russias',
  '/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels/russias/russias__LJ042-0204__occ1__seqlen38.pt'),
 ('techniques',
  '/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels/techniques/techniques__LJ046-0157__occ1__seqlen73.pt'),
 ('techniques',
  '/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels/techniques/techniques__LJ050-0196__occ1__seqlen59.pt')]

## process text

In [50]:
from fastpitch.common.text.text_processing import TextProcessor
text_cleaners = ['lowercase_no_punc']
symbol_set = "english_pad_lowercase_nopunc"
tp = TextProcessor(symbol_set, text_cleaners, add_spaces=False, eos_symbol="$")

encoded = torch.IntTensor(tp.encode_text("JASON's!!! How?! do yyyou-identifies??.   "))
encoded

tensor([11,  2, 20, 16, 15, 20,  1,  9, 16, 24,  1,  5, 16,  1, 26, 26, 26, 16,
        22,  1, 10,  5,  6, 15, 21, 10,  7, 10,  6, 20], dtype=torch.int32)

In [51]:
decoded = [tp.id_to_symbol[id] for id in encoded.tolist()]
"".join(decoded)

'jasons how do yyyou identifies'

## process mel

In [52]:
word, fp = token_and_melfilepaths[0]
wordaligned_mel = torch.load(fp)
wordaligned_mel.size() # [seqlen, feats]

torch.Size([68, 80])

## 'class'-ified dataset class

In [53]:
from fastpitch.common.text.text_processing import TextProcessor

class RespellerDataset(torch.utils.data.Dataset):
    """
        1) loads word + word-aligned mel spec for all words in a wordlist
        2) converts text to sequences of one-hot vectors (corresponding to grapheme indices in fastpitch)
    """
    def __init__(
        self, 
        wordaligned_speechreps_dir, # path to directory that contains folders of word aligned speech reps
        wordlist, # txt file for the words to include speech reps from
        max_examples_per_wordtype=None,
        text_cleaners=[],
        symbol_set="english_basic_lowercase_no_arpabet",
        add_spaces=True,
        eos_symbol="$",
        **kwargs,
    ):
        # load wordlist as a python list
        if type(wordlist) == str:
            if wordlist.endswith('.json'):
                with open(wordlist) as f:
                    wordlist = json.load(f)
            else:
                with open(wordlist) as f:
                    wordlist = f.read().splitlines()
        elif type(wordlist) == list:
            pass # dont need to do anything, already in expected form
        elif type(wordlist) == set:
            wordlist = list(wordlist)
        
        wordlist = sorted(wordlist)
        
        # create list of all word tokens and their word aligned speech reps
        self.word_freq = Counter()
        self.token_and_melfilepaths = []
        print("Initialising respeller dataset")
        for word in tqdm(wordlist):
            # find all word aligned mels for the word
            word_dir = os.path.join(wordaligned_speechreps_dir, word)
            mel_files = os.listdir(word_dir)
            if max_examples_per_wordtype:
                mel_files = mel_files[:max_examples_per_wordtype]
            for mel_file in mel_files:
                mel_file_path = os.path.join(word_dir, mel_file)
                self.token_and_melfilepaths.append((word, mel_file_path))
                self.word_freq[word] += 1
                
        self.tp = TextProcessor(symbol_set, text_cleaners, add_spaces=add_spaces, eos_symbol=eos_symbol)

    def get_mel(self, filename):
        return torch.load(filename)

    def encode_text(self, text):
        """encode raw text into indices defined by grapheme embedding table of the TTS model"""
        return torch.IntTensor(self.tp.encode_text(text))
    
    def decode_text(self, encoded):
        # if encoded.dim() == 1:
        #     decodings = [self.tp.id_to_symbol[id] for id in encoded.tolist()]
        # else:
        decodings = []
        for batch_idx in range(encoded.size(0)):
            decodings.append(''.join(self.tp.id_to_symbol[idx] for idx in encoded[batch_idx].tolist()))
        return decodings
    
    @staticmethod
    def get_mel_len(melfilepath):
        return int(melfilepath.split('seqlen')[1].split('.pt')[0])

    def __getitem__(self, index):
        word, mel_filepath = self.token_and_melfilepaths[index]
        encoded_word = self.encode_text(word)
        mel = self.get_mel(mel_filepath)
        
        return {
            'word': word, 
            'encoded_word': encoded_word, 
            'mel_filepath': mel_filepath,
            'mel': mel,
        }

    def __len__(self):
        return len(self.token_and_melfilepaths)

In [54]:
dataset = RespellerDataset(
    wordaligned_speechreps_dir='/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels', # path to directory that contains folders of word aligned speech reps
    wordlist=['identifies','mash','player','russias','techniques'],
)

Initialising respeller dataset


100%|█████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 1361.70it/s]


In [55]:
batch = []

for itemdict in dataset:
    # unpack dict
    word = itemdict['word'] 
    encoded_word = itemdict['encoded_word'] 
    mel = itemdict['mel'] 
    
    # check
    print(word, encoded_word, mel.size())
    
    batch.append(itemdict)

identifies tensor([12, 21, 16, 17, 26, 32, 21, 18, 21, 17, 31, 39], dtype=torch.int32) torch.Size([68, 80])
mash tensor([12, 25, 13, 31, 20, 39], dtype=torch.int32) torch.Size([28, 80])
player tensor([12, 28, 24, 13, 37, 17, 30, 39], dtype=torch.int32) torch.Size([49, 80])
russias tensor([12, 30, 33, 31, 31, 21, 13, 31, 39], dtype=torch.int32) torch.Size([38, 80])
techniques tensor([12, 32, 17, 15, 20, 26, 21, 29, 33, 17, 31, 39], dtype=torch.int32) torch.Size([73, 80])
techniques tensor([12, 32, 17, 15, 20, 26, 21, 29, 33, 17, 31, 39], dtype=torch.int32) torch.Size([59, 80])


## collate function

In [56]:
class Collate():
    """ Zero-pads model inputs and targets based on number of frames per setep
    """
    # def __init__(self):
    
    def __call__(self, batch):
        """Collate's training batch from encoded word token and its 
        corresponding word-aligned mel spectrogram
        
        batch: [encoded_token, wordaligned_mel]
        """
        # Right zero-pad all one-hot text sequences to max input length
        input_lengths, ids_sorted_decreasing = torch.sort(
            torch.LongTensor([len(x['encoded_word']) for x in batch]),
            dim=0, descending=True)
        max_input_len = input_lengths[0]

        words = []
        mel_filepaths = []
        text_padded = torch.LongTensor(len(batch), max_input_len)
        text_padded.zero_()
        text_lengths = torch.LongTensor(len(batch))
        for i in range(len(ids_sorted_decreasing)):
            words.append(batch[ids_sorted_decreasing[i]]['word'])
            mel_filepaths.append(batch[ids_sorted_decreasing[i]]['mel_filepath'])
            text = batch[ids_sorted_decreasing[i]]['encoded_word']
            text_padded[i, :text.size(0)] = text
            text_lengths[i] = text.size(0)

        # Right zero-pad mel-spec
        num_mels = batch[0]['mel'].size(1)
        max_target_len = max([x['mel'].size(0) for x in batch])

        mel_padded = torch.FloatTensor(len(batch), max_target_len, num_mels)
        mel_padded.zero_()
        mel_lengths = torch.LongTensor(len(batch))
        for i in range(len(ids_sorted_decreasing)):
            mel = batch[ids_sorted_decreasing[i]]['mel']
            mel_padded[i, :mel.size(0), :] = mel
            mel_lengths[i] = mel.size(0)
            

        return {
            'words': words,
            'text_padded': text_padded,
            'text_lengths': text_lengths,
            'mel_padded': mel_padded, 
            'mel_lengths': mel_lengths,
            'mel_filepaths': mel_filepaths
        }
                # input_lengths, mel_padded, output_lengths,
                # len_x, dur_padded, dur_lens, pitch_padded, speaker)

In [57]:
collate_fn = Collate()
collated = collate_fn(batch)

In [58]:
collated['text_padded'].size()

torch.Size([6, 12])

In [59]:
collated['text_padded']

tensor([[12, 21, 16, 17, 26, 32, 21, 18, 21, 17, 31, 39],
        [12, 32, 17, 15, 20, 26, 21, 29, 33, 17, 31, 39],
        [12, 32, 17, 15, 20, 26, 21, 29, 33, 17, 31, 39],
        [12, 30, 33, 31, 31, 21, 13, 31, 39,  0,  0,  0],
        [12, 28, 24, 13, 37, 17, 30, 39,  0,  0,  0,  0],
        [12, 25, 13, 31, 20, 39,  0,  0,  0,  0,  0,  0]])

In [60]:
collated['words']

['identifies', 'techniques', 'techniques', 'russias', 'player', 'mash']

In [61]:
collated['mel_padded'].size()

torch.Size([6, 73, 80])

## put batch on gpu

In [62]:
def to_gpu(x):
    x = x.contiguous()
    if torch.cuda.is_available():
        x = x.cuda(non_blocking=True)
    return torch.autograd.Variable(x)

def batch_to_gpu(collated_batch):
    """put elements that are used throughout training onto gpu"""
    words = collated_batch['words']
    text_padded = collated_batch['text_padded']
    text_lengths = collated_batch['text_lengths']
    mel_padded = collated_batch['mel_padded']
    mel_lengths = collated_batch['mel_lengths']
    
    # no need to put words on gpu, its only used during eval loop
    text_padded = to_gpu(text_padded).long()
    text_lengths = to_gpu(text_lengths).long()
    mel_padded = to_gpu(mel_padded).float()
    mel_lengths = to_gpu(mel_lengths).long()
    
    # x: inputs
    x = {
        'words': words,
        'text_padded': text_padded,
        'text_lengths': text_lengths,
    }
    # y: targets
    y = {
        'mel_padded': mel_padded, 
        'mel_lengths': mel_lengths,
    }
    
    return (x, y)

In [63]:
# batch_to_gpu(collated)

# full train + dev datasets

In [64]:
train_dataset = RespellerDataset(
    wordaligned_speechreps_dir=args.wordaligned_speechreps, # path to directory that contains folders of word aligned speech reps
    wordlist=args.train_wordlist,
    max_examples_per_wordtype=args.max_examples_per_wordtype,
    add_spaces=args.add_spaces,
)

Initialising respeller dataset


100%|███████████████████████████████████████████████████████████████████████████████| 7479/7479 [00:04<00:00, 1514.78it/s]


In [65]:
len(train_dataset)

7479

In [66]:
sum(train_dataset.word_freq.values())

7479

In [67]:
val_dataset = RespellerDataset(
    wordaligned_speechreps_dir=args.wordaligned_speechreps, # path to directory that contains folders of word aligned speech reps
    wordlist=args.val_wordlist,
    add_spaces=args.add_spaces,
)

Initialising respeller dataset


100%|█████████████████████████████████████████████████████████████████████████████████| 415/415 [00:00<00:00, 1572.58it/s]


In [68]:
len(val_dataset)

415

In [69]:
sum(val_dataset.word_freq.values())

415

# create torch dataloader

In [70]:
# TODO, implement distributed training?
train_sampler = None
shuffle = True
num_cpus = 1 
train_loader = DataLoader(train_dataset, num_workers=2*num_cpus, shuffle=shuffle,
                          sampler=train_sampler, batch_size=args.batch_size,
                          pin_memory=False, drop_last=True,
                          collate_fn=collate_fn)

In [71]:
# for batch in train_loader:
#     print(batch)

# FULL train() loop

## init dl logger

In [72]:
def touch_file(path):
    if not os.path.exists(path):
        basedir = os.path.dirname(path)
        os.makedirs(basedir, exist_ok=True)
        with open(path, 'w') as f:
            f.write("")

# initialise logger
tb_subsets = ['train', 'val']
log_fpath = args.log_file or os.path.join(args.chkpt_save_dir, 'nvlog.json')
touch_file(log_fpath)

try: 
    logger.init(log_fpath, args.chkpt_save_dir, enabled=(args.local_rank == 0),
                tb_subsets=tb_subsets)
    logger.parameters(vars(args), tb_subset='train')
except:
    print("WARNING DLLLoggerAlreadyInitialized error raised")

DLL 2023-01-24 15:07:44 - PARAMETER | wandb_project_name:  respeller-training
DLL 2023-01-24 15:07:44 - PARAMETER | chkpt_save_dir:  /home/s1785140/respeller/exps/ConcatPosEncodingAndDropoutLayer
DLL 2023-01-24 15:07:44 - PARAMETER | dataset_path:  ./
DLL 2023-01-24 15:07:44 - PARAMETER | log_file: 
DLL 2023-01-24 15:07:44 - PARAMETER | cuda:  True
DLL 2023-01-24 15:07:44 - PARAMETER | batch_size:  4
DLL 2023-01-24 15:07:44 - PARAMETER | val_num_to_gen:  2
DLL 2023-01-24 15:07:44 - PARAMETER | seed:  1337
DLL 2023-01-24 15:07:44 - PARAMETER | grad_accumulation:  1
DLL 2023-01-24 15:07:44 - PARAMETER | epochs:  2
DLL 2023-01-24 15:07:44 - PARAMETER | max_iters_per_epoch:  5
DLL 2023-01-24 15:07:44 - PARAMETER | epochs_per_checkpoint:  10
DLL 2023-01-24 15:07:44 - PARAMETER | checkpoint_path: 
DLL 2023-01-24 15:07:44 - PARAMETER | resume:  False
DLL 2023-01-24 15:07:44 - PARAMETER | val_log_interval:  1
DLL 2023-01-24 15:07:44 - PARAMETER | speech_length_penalty_training:  False
DLL 2023

## functions

In [73]:
def adjust_learning_rate(total_iter, opt, learning_rate, warmup_iters=None):
    if warmup_iters == 0:
        scale = 1.0
    elif total_iter > warmup_iters:
        scale = 1. / (total_iter ** 0.5)
    else:
        scale = total_iter / (warmup_iters ** 1.5)

    for param_group in opt.param_groups:
        param_group['lr'] = learning_rate * scale

def log_stdout(logger, subset, epoch_iters, total_steps, loss, took):
    logger_data = [
        ('Loss/Total', loss),
    ]
    logger_data.append(('Time/Iter time', took))
    logger.log(epoch_iters,
               tb_total_steps=total_steps,
               subset=subset,
               data=OrderedDict(logger_data)
    )

def maybe_save_checkpoint(args, model, optimizer, epoch,
                          total_iter, config):
    if args.local_rank != 0:
        return

    intermediate = (args.epochs_per_checkpoint > 0
                    and epoch % args.epochs_per_checkpoint == 0)

    if not intermediate and epoch < args.epochs:
        return

    fpath = os.path.join(args.chkpt_save_dir, f"respeller_checkpoint_{epoch}.pt")
    print(f"Saving model and optimizer state at epoch {epoch} to {fpath}")
    checkpoint = {'epoch': epoch,
                  'iteration': total_iter,
                  'config': config,
                  'state_dict': model.state_dict(),
                  'optimizer': optimizer.state_dict()}
    torch.save(checkpoint, fpath)

def calc_sl_penalty(pred_lens, gt_lens):
    '''speech length mismatch penalty similar to MCD-DTW-SL
    encourages two sequences to be of same length
    M and N are length of each sequence
    coef = Max(M,N) / Min(M,N)'''
    # stack so we can calculate max along batch dimension
    stacked = torch.stack([pred_lens, gt_lens])
    maxs, _ = torch.max(stacked, dim=0)
    mins, _ = torch.min(stacked, dim=0)
    coefs = maxs/mins
    return coefs

## pre-training loop stuff

In [74]:
# def train(rank, args):


# handle GPU
rank = 0
args.local_rank = rank
device = torch.device('cuda' if args.cuda else 'cpu')

# load models
tts, n_symbols, grapheme_embedding_dim, model_config = load_pretrained_fastpitch(args)
respeller = EncoderRespeller(n_symbols=n_symbols, pretrained_tts=tts, d_model=args.d_model)
# quantiser = GumbelVectorQuantizer(
#     in_dim=args.d_model,
#     codebook_size=n_symbols,  # number of codebook entries
#     embedding_dim=grapheme_embedding_dim,
#     temp=args.latent_temp,
# )
# init_embedding_weights(tts.encoder.word_emb.weight.unsqueeze(0), quantiser.vars)
def mean_absolute_error(x, y):
    """for calculating softdtw using L1 loss
    Calculates the Euclidean distance between each element in x and y per timestep
    """
    n = x.size(1)
    m = y.size(1)
    d = x.size(2)
    x = x.unsqueeze(2).expand(-1, n, m, d)
    y = y.unsqueeze(1).expand(-1, n, m, d)
    return torch.abs(x - y).sum(3)

if args.dist_func == 'l1':
    dist_func = mean_absolute_error
elif args.dist_func == 'l2':
    dist_func = None # softdtw package uses L2 as default
    
criterion = SoftDTW(use_cuda=True, gamma=args.softdtw_temp, bandwidth=args.softdtw_bandwidth, dist_func=dist_func) # input should be size [bsz, seqlen, dim]

tts.to(device)
respeller.to(device)
# quantiser.to(device)
criterion.to(device)

# load optimiser and assign to it the weights to be trained
kw = dict(lr=args.learning_rate, betas=(0.9, 0.98), eps=1e-9,
          weight_decay=args.weight_decay)
optimizer = Lamb(respeller.trainable_parameters(), **kw)

# (optional) load checkpoint for respeller
start_epoch = [1]
start_iter = [0]
assert args.checkpoint_path is None or args.resume is False, (
    "Specify a single checkpoint source")
if args.checkpoint_path is not None:
    ch_fpath = args.checkpoint_path
elif args.resume:
    ch_fpath = last_checkpoint(args.chkpt_save_dir)
else:
    ch_fpath = None
if ch_fpath is not None:
    load_respeller_checkpoint(args, respeller, ch_fpath, optimizer, start_epoch, start_iter)
    
start_epoch = start_epoch[0]
total_iter = start_iter[0]
    
# create datasets, collate func, dataloader
train_dataset = RespellerDataset(
    wordaligned_speechreps_dir='/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels', # path to directory that contains folders of word aligned speech reps
    wordlist='/home/s1785140/data/ljspeech_fastpitch/respeller_train_words.json',
    max_examples_per_wordtype=2,
    symbol_set="english_basic_lowercase_no_arpabet",
    add_spaces=args.add_spaces,
)
val_dataset = RespellerDataset(
    wordaligned_speechreps_dir='/home/s1785140/data/ljspeech_fastpitch/wordaligned_mels', # path to directory that contains folders of word aligned speech reps
    wordlist='/home/s1785140/data/ljspeech_fastpitch/respeller_dev_words.json',
    symbol_set="english_basic_lowercase_no_arpabet",
    add_spaces=args.add_spaces,
)
num_cpus = 1 # TODO change to CLA?
train_loader = DataLoader(train_dataset, num_workers=2*num_cpus, shuffle=True,
                          sampler=None, batch_size=args.batch_size,
                          pin_memory=False, drop_last=True,
                          collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, num_workers=2*num_cpus, shuffle=False,
                          sampler=None, batch_size=args.batch_size,
                          pin_memory=False, collate_fn=collate_fn)

# load pretrained hifigan

# log spectrograms and generated audio for first few validation wordtypes

# train loop
respeller.train()
# quantiser.train()
tts.eval()

print('Finished setting up models + dataloaders')

Loading model and optimizer state from fastpitch/exps/halved_ljspeech_data_nospaces_noeos_pad_lowercase_nopunc/FastPitch_checkpoint_1000.pt
Initialising respeller dataset


100%|████████████████████████████████████████████████████████████████████████████████| 7479/7479 [00:11<00:00, 657.73it/s]


Initialising respeller dataset


100%|█████████████████████████████████████████████████████████████████████████████████| 415/415 [00:00<00:00, 1609.52it/s]

Finished setting up models + dataloaders


## validate()

### plot spectrogram

In [75]:
def log_spectrogram(log_mel, figsize=(15,5), image_name=""):
    fig, ax = plt.subplots(figsize=figsize)
    img = librosa.display.specshow(log_mel, ax=ax, x_axis='frames', y_axis='linear')
    ax.set_title(image_name)
    fig.colorbar(img, ax=ax)
    return fig

def get_spectrograms_plots(y, fnames, step, n=4, label='Predicted spectrogram', mas=False, return_figs=False):
    """Plot spectrograms for n utterances in batch"""
    bs = len(fnames)
    n = min(n, bs)
    s = bs // n
    fnames = fnames[::s]
    # print(f"inside get_spectrograms_plots(), {fnames=}")
    if label == 'Predicted spectrogram':
        # y: mel_out, dec_mask, dur_pred, log_dur_pred, pitch_pred
        mel_specs = y[0][::s].transpose(1, 2).cpu().numpy()
        mel_lens = y[1][::s].cpu().numpy() - 1
    elif label == 'Reference spectrogram':
        # y: mel_padded, mel_lens
        mel_specs = y[0][::s].cpu().numpy()
        # if mas:
        mel_lens = y[1][::s].cpu().numpy()  # output_lengths
        # else:
            # mel_lens = y[1][::s].cpu().numpy().sum(axis=1) - 1
            
    image_names = []
    spectrograms = []
    for mel_spec, mel_len, fname in zip(mel_specs, mel_lens, fnames):
        mel_spec = mel_spec[:, :mel_len]
        utt_id = os.path.splitext(os.path.basename(fname))[0]
        image_name = f'val/{label}/{utt_id}'
        fig = log_spectrogram(mel_spec, image_name=image_name)
        image_names.append(image_name)
        
        if return_figs:
            spectrograms.append(fig)
        else:
            buf = BytesIO()
            fig.savefig(buf, format='png')
            img = Image.open(buf)
            plt.close(fig)
            spectrograms.append(img)
            
    return image_names, spectrograms

### gen audio

In [76]:
def generate_audio(y, fnames, step, vocoder=None, sampling_rate=22050, hop_length=256,
                   n=4, label='Predicted audio', mas=False):
    """Generate audio from spectrograms for n utterances in batch"""
    bs = len(fnames)
    n = min(n, bs)
    s = bs // n
    fnames = fnames[::s]
    # print(f"inside generate_audio(), {fnames=}")
    with torch.no_grad():
        if label == 'Predicted audio':
            # y: mel_out, dec_mask, dur_pred, log_dur_pred, pitch_pred
            audios = vocoder(y[0][::s].transpose(1, 2)).cpu().squeeze(1).numpy() # [bsz, dim, samples ]only squeeze away dim (equals 1 for waveform)
            # print(f"{vocoder(y[0][::s].transpose(1, 2)).cpu().size()=}")
            # print(f"{vocoder(y[0][::s].transpose(1, 2)).cpu().squeeze().size()=}")
            # print(f"{y[1][::s]=}, {y[1][::s].size()=}")
            # print(f"{y[1][::s].squeeze()=}, {y[1][::s].squeeze().size()=}")
            mel_lens = y[1][::s].cpu().numpy() - 1
        elif label == 'Copy synthesis':
            # y: mel_padded, dur_padded, dur_lens, pitch_padded
            audios = vocoder(y[0][::s]).cpu().squeeze().numpy()
            if mas:
                mel_lens = y[2][::s].cpu().numpy()  # output_lengths
            else:
                mel_lens = y[1][::s].cpu().numpy().sum(axis=1) - 1
        elif label == 'Reference audio':
            audios = []
            for fname in fnames:
                wav = re.sub(r'mels/(.+)\.pt', r'wavs/\1.wav', fname)
                audio, _ = librosa.load(wav, sr=sampling_rate)
                audios.append(audio)
            if mas:
                mel_lens = y[2][::s].cpu().numpy()  # output_lengths
            else:
                mel_lens = y[1][::s].cpu().numpy().sum(axis=1) - 1
    audios_to_return = []
    # print(f"DEBUG generate_audio(), {type(audios)=} {type(mel_lens)=} {type(fnames)=}")
    # print(f"DEBUG generate_audio(), {audios.shape=} {mel_lens.shape=} {len(fnames)=}")
    for audio, mel_len, fname in zip(audios, mel_lens, fnames):
        audio = audio[:mel_len * hop_length]
        audio = audio / np.max(np.abs(audio))
        utt_id = os.path.splitext(os.path.basename(fname))[0]
        audios_to_return.append(audio)
        
    return audios_to_return

### create and log a wandb table (class)

In [77]:
class WandbTable:
    def __init__(self):  
        self.table = wandb.Table(columns=[
            "names", 
            "orig spelling", 
            "orig spelling spec", 
            "orig spelling audio",
            "vocoded gt spec",
            "vocoded gt audio",
            "respelling", 
            "respelling spec", 
            "respelling audio",
            "sl penalty coef",
            "softdtw loss",
        ])
        
    def add_rows(
        self,
        names, 
        vocoded_gt_specs,
        vocoded_gt_audios,
        orig_words,
        respellings,
        orig_pred_specs, # either PIL images or matplotlib figures (but might have mem issues!)
        orig_pred_audios,
        pred_specs, # either PIL images or matplotlib figures (but might have mem issues!)
        pred_audios,
        sl_penalty_coefs,
        losses,
        sampling_rate=22050,
    ):
        for (
            name, 
            orig_word, 
            orig_pred_spec_fig, 
            orig_pred_audio, 
            vocoded_gt_spec_fig,
            vocoded_gt_audio, 
            respelling, 
            pred_spec_fig, 
            pred_audio,
            sl_penalty_coef,
            loss,
        ) in zip(
            names, 
            orig_words, 
            orig_pred_specs,
            orig_pred_audios,
            vocoded_gt_specs,
            vocoded_gt_audios,
            respellings, 
            pred_specs, 
            pred_audios,
            sl_penalty_coefs,
            losses,
        ):
            self.table.add_data(
                name, 
                orig_word,
                wandb.Image(orig_pred_spec_fig, caption=name),
                wandb.Audio(orig_pred_audio, caption=name, sample_rate=sampling_rate),
                wandb.Image(vocoded_gt_spec_fig, caption=name),
                wandb.Audio(vocoded_gt_audio, caption=name, sample_rate=sampling_rate),
                respelling,
                wandb.Image(pred_spec_fig, caption=name),
                wandb.Audio(pred_audio, caption=name, sample_rate=sampling_rate),
                sl_penalty_coef,
                loss,
            )
            
            # close figures to save memory
            if type(orig_pred_spec_fig) == matplotlib.figure.Figure:
                plt.close(orig_pred_spec_fig)
            if type(vocoded_gt_spec_fig) == matplotlib.figure.Figure:
                plt.close(vocoded_gt_spec_fig)
            if type(pred_spec_fig) == matplotlib.figure.Figure:
                plt.close(pred_spec_fig)
            
    def log(self, train):
        if train:
            wandb.log({"train_table": self.table})
        else:
            wandb.log({"val_table": self.table})

### validate() fn

In [78]:
def decode_indices(indices):
    """decode batch of indices to text
    [bsz, seqlen]"""
    decodings = []
    for batch_idx in range(indices.size(0)):
        decodings.append(''.join(tp.id_to_symbol[id] for id in indices[batch_idx].tolist()))
    return decodings

def select(x, bsz, n):
    """select items in batch that will be visualised/converted to audio"""
    n = min(n, bsz)
    s = bsz // n
    return x[::s]

def validate(
    respeller_model, 
    tts_model, 
    vocoder,
    criterion,
    valset, 
    epoch, 
    batch_size, 
    num_to_gen,
    collate_fn, 
    sampling_rate,
    hop_length,
    audio_interval=5,
    only_log_table=False,
    train=False,
):
    """Handles all the validation scoring and printing
    GT (beginning of training):
    - log GT mel spec and vocoded audio for several validation set words
    
    Model outputs:
    - log predicted mel spec and vocoded audio from fastpitch
    - log respelled word from respeller
    """
    was_training = respeller_model.training
    respeller_model.eval()
    wandb_table = WandbTable()
    
    tik = time.perf_counter()
    with torch.no_grad():
        val_loader = DataLoader(valset, num_workers=4, shuffle=False,
                                sampler=None,
                                batch_size=batch_size, pin_memory=False,
                                collate_fn=collate_fn)
        val_meta = defaultdict(float)
        val_losses = 0.0
        val_losses_with_sl_penalty = 0.0
        epoch_iter = 0
        sl_penalty_coefs = []
        
        num_generated = 0
        
        for i, batch in enumerate(val_loader):
            epoch_iter += 1
            
            # get loss over batch
            x, y = batch_to_gpu(batch)
            pred_mel, dec_lens, g_embedding_indices = forward_pass(respeller_model, tts_model, x)
            iter_loss = (criterion(pred_mel, y["mel_padded"]))
            val_losses += iter_loss.mean().item()
            
            coef = calc_sl_penalty(dec_lens, y['mel_lengths'])
            sl_penalty_coefs.append(coef.mean().item())
            val_losses_with_sl_penalty += (coef * iter_loss).mean().item()
    
            # log spectrograms and generated audio for first few utterances
            log_table = (epoch % audio_interval == 0 if epoch is not None else True)
            
            should_generate = num_generated < args.val_num_to_gen
            if log_table and should_generate:
                fnames = batch['mel_filepaths']
                bsz = len(fnames)
                
                num_to_generate_this_batch = min(bsz, args.val_num_to_gen - num_generated)
                
                # print(f"DEBUG validate(), {bsz=}, {num_to_generate_this_batch=}")
                
                # get original word and respellings for logging
                original_words = valset.decode_text(x['text_padded'])
                respellings = valset.decode_text(g_embedding_indices)
                
                
                # vocode original recorded speech
                gt_mel = y['mel_padded']
                gt_mel_lens = y['mel_lengths']
                vocoded_gt = generate_audio((gt_mel, gt_mel_lens), fnames, total_iter, vocoder, 
                                            sampling_rate, hop_length, n=num_to_generate_this_batch, 
                                            label='Predicted audio', mas=True)
                _orig_token_names, gt_specs = get_spectrograms_plots(
                    (gt_mel.transpose(1,2), gt_mel_lens), fnames, total_iter, 
                    n=num_to_generate_this_batch, label='Reference spectrogram', mas=False)
                
                # get melspec + generated audio for original spellings
                orig_pred_mel, orig_dec_lens, _dur_pred, _pitch_pred = tts(
                    inputs=x['text_padded'],
                    skip_embeddings=False,
                )
                orig_pred_mel = orig_pred_mel.transpose(1,2)
                _orig_token_names, orig_pred_specs = get_spectrograms_plots(
                    (orig_pred_mel, orig_dec_lens), fnames, total_iter, 
                    n=num_to_generate_this_batch, label='Predicted spectrogram', mas=True)
                orig_pred_audios = generate_audio((orig_pred_mel, orig_dec_lens), fnames, 
                                                  total_iter, vocoder, sampling_rate, hop_length, 
                                                  n=num_to_generate_this_batch, label='Predicted audio', mas=True)
            
                # get melspec + generated audio for respellings
                token_names, pred_specs = get_spectrograms_plots(
                    (pred_mel, dec_lens), fnames, total_iter, 
                    n=num_to_generate_this_batch, label='Predicted spectrogram', mas=True)
                pred_audios = generate_audio(
                    (pred_mel, dec_lens), fnames, total_iter, vocoder, sampling_rate, hop_length, 
                    n=num_to_generate_this_batch, label='Predicted audio', mas=True)
                
                # log everything to wandb table
                token_names = [tok_name.split('/')[-1] for tok_name in token_names]
                
                wandb_table.add_rows(
                    names=token_names,
                    vocoded_gt_specs=gt_specs,
                    vocoded_gt_audios=vocoded_gt,
                    orig_words=select(original_words, bsz, n=num_to_generate_this_batch),
                    orig_pred_specs=orig_pred_specs,
                    orig_pred_audios=orig_pred_audios,
                    respellings=select(respellings, bsz, n=num_to_generate_this_batch),
                    pred_specs=pred_specs,
                    pred_audios=pred_audios,
                    sl_penalty_coefs=coef,
                    losses=iter_loss,
                    sampling_rate=sampling_rate,
                )
                
                num_generated += num_to_generate_this_batch
                
            if train or log_table and only_log_table:
                break # leave for loop after first iteration
        
        if not only_log_table:
            val_logs = {}
            val_logs['val/epoch_loss'] = val_losses/epoch_iter
            if val_losses_with_sl_penalty != 0.0:
                val_logs['val/epoch_loss_with_sl_penalty'] = val_losses_with_sl_penalty/epoch_iter
                val_logs['val/epoch_sl_penalty_coef'] = sum(sl_penalty_coefs) / len(sl_penalty_coefs)
            wandb.log(val_logs)
    
    wandb_table.log(train=train)
    
    if was_training:
        respeller_model.train()

## forward pass

### dev

In [79]:
# test forward_pass

# get one batch
batch = next(iter(train_loader))
x, y = batch_to_gpu(batch)

In [80]:
# get src key padding mask
def get_src_key_padding_mask(bsz, max_len, lens, device):
    """return a Boolean mask for a list or tensor of sequence lengths
    True for values in tensor greater than sequence length

    bsz (int)
    max_len (int): max seq len of item in batch
    lens [bsz]: list or tensor of lengths
    """
    if type(lens) == list:
        lens = torch.tensor(lens, device=device)
    assert lens.dim() == 1

    lens = lens.unsqueeze(1)  # [bsz] -> [bsz, seq_len]
    m = torch.arange(max_len, device=device)
    m = m.expand(bsz, max_len)  # repeat along batch dimension
    m = (m < lens)
    return ~m  # tilde inverts a bool tensor

text_lens = x['text_lengths']
max_len = max(text_lens).item()
bsz = len(text_lens)

mask = get_src_key_padding_mask(bsz, max_len, text_lens, x['text_padded'].device)

In [81]:
g_embeddings, g_embedding_indices = respeller(x['text_padded'], mask)

debug respellerforward() enc_inputs.size()=torch.Size([4, 11, 384]) pos_seq.size()=torch.Size([11]) inverted_mask.size()=torch.Size([4, 11])


/opt/conda/conda-bld/pytorch_1659484657607/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [30,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1659484657607/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [30,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1659484657607/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [30,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1659484657607/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [30,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1659484657607/work/aten/src/ATen/native/cuda/Indexing.cu:975: indexSelectLargeIndex: block: [30,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld

RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`

In [ ]:
# optionally pad outputs according to the text_len/padding in the input original spellings
# NB do this in the forward of the respeller model?


In [ ]:
log_mel, dec_lens, _dur_pred, _pitch_pred = tts(
    inputs=g_embeddings,
    ids=g_embedding_indices,
    skip_embeddings=True,
)

# log_mel: [bsz, dim, seqlen]
log_mel = log_mel.transpose(1,2)

### func definition

In [ ]:
def forward_pass(respeller, tts, x):
    """x: inputs
    x = {
        'words': words,
        'text_padded': text_padded,
        'text_lengths': text_lengths,
    }"""
    text_lens = x['text_lengths']
    max_len = max(text_lens).item()
    bsz = len(text_lens)

    mask = get_src_key_padding_mask(bsz, max_len, text_lens, x['text_padded'].device)
    
    g_embeddings, g_embedding_indices = respeller(x['text_padded'], mask)
    
    # quantiser_outdict = quantiser(logits, produce_targets=True)
    # g_embedding_indices = quantiser_outdict["targets"].squeeze(2)
    # g_embeddings = quantiser_outdict["x"]
    
    # use text lens to zero out the output of the respeller so that repelling matches the length of the original spelling
    for i, text_len in enumerate(text_lens):
        # ###OLD
        # #indices
        # indices = g_embedding_indices[i, :text_len]
        # masked_g_embedding_indices[i, :text_len] = indices
        # #embeddings
        # embeddings = g_embeddings[i, :text_len, :]
        # masked_g_embeddings[i, :text_len, :] = embeddings
        ###NEW
        g_embedding_indices[i, text_len:] = 0.0
        g_embeddings[i, text_len:, :] = 0.0
    

    log_mel, dec_lens, _dur_pred, _pitch_pred = tts(
        inputs=g_embeddings,
        ids=g_embedding_indices,
        skip_embeddings=True,
    )
    
    # log_mel: [bsz, dim, seqlen]
    log_mel = log_mel.transpose(1,2)
    # log_mel: [bsz, seqlen, dim]
    
    # return mask for masking acoustic loss
    # padding_idx = 0
    # mask = (g_embedding_indices != padding_idx).unsqueeze(2)
    # mask.size()
    # dec_mask = mask_from_lens(dec_lens).unsqueeze(2)
    
    return log_mel, dec_lens, g_embedding_indices

def byte_to_gigabyte(bytes):
    return bytes/1000000000

def run_val(epoch):
    """wrap in fn so that we can call at:
    1. before training model
    2. at end of every X epochs"""
    # log audio and respellings for training set words
    validate(
        respeller_model=respeller, 
        tts_model=tts, 
        vocoder=vocoder,
        criterion=criterion,
        valset=train_dataset, 
        batch_size=args.batch_size,
        num_to_gen=args.val_num_to_gen,
        collate_fn=collate_fn,
        epoch=epoch,
        sampling_rate=args.sampling_rate,
        hop_length=args.hop_length,
        audio_interval=args.val_log_interval,
        only_log_table=True,
        train=True,
    )
        
    # log audio and respellings for val set words
    validate(
        respeller_model=respeller, 
        tts_model=tts, 
        vocoder=vocoder,
        criterion=criterion,
        valset=val_dataset, 
        batch_size=args.batch_size,
        num_to_gen=args.val_num_to_gen,
        collate_fn=collate_fn,
        epoch=epoch,
        sampling_rate=args.sampling_rate,
        hop_length=args.hop_length,
        audio_interval=args.val_log_interval,
    )

## before train loop validation

DEBUG validate(), bsz=1, num_to_generate_this_batch=1
y[1][::s]=tensor([43], device='cuda:0'), y[1][::s].size()=torch.Size([1])
y[1][::s].squeeze()=tensor(43, device='cuda:0'), y[1][::s].squeeze().size()=torch.Size([])
DEBUG generate_audio(), type(audios)=<class 'numpy.ndarray'> type(mel_lens)=<class 'numpy.int64'> type(fnames)=<class 'list'>
DEBUG generate_audio(), audios.shape=(11008,) mel_lens.shape=() len(fnames)=1

DEBUG validate(), bsz=2, num_to_generate_this_batch=2
vocoder(y[0][::s].transpose(1, 2)).cpu().size()=torch.Size([2, 1, 11264])
vocoder(y[0][::s].transpose(1, 2)).cpu().squeeze().size()=torch.Size([2, 11264])
y[1][::s]=tensor([43, 44], device='cuda:0'), y[1][::s].size()=torch.Size([2])
y[1][::s].squeeze()=tensor([43, 44], device='cuda:0'), y[1][::s].squeeze().size()=torch.Size([2])
DEBUG generate_audio(), type(audios)=<class 'numpy.ndarray'> type(mel_lens)=<class 'numpy.ndarray'> type(fnames)=<class 'list'>
DEBUG generate_audio(), audios.shape=(2, 11264) mel_lens.shape=(2,) len(fnames)=2
vocoder(y[0][::s].transpose(1, 2)).cpu().size()=torch.Size([2, 1, 17152])
vocoder(y[0][::s].transpose(1, 2)).cpu().squeeze().size()=torch.Size([2, 17152])
y[1][::s]=tensor([67, 67], device='cuda:0'), y[1][::s].size()=torch.Size([2])
y[1][::s].squeeze()=tensor([67, 67], device='cuda:0'), y[1][::s].squeeze().size()=torch.Size([2])
DEBUG generate_audio(), type(audios)=<class 'numpy.ndarray'> type(mel_lens)=<class 'numpy.ndarray'> type(fnames)=<class 'list'>
DEBUG generate_audio(), audios.shape=(2, 17152) mel_lens.shape=(2,) len(fnames)=2
vocoder(y[0][::s].transpose(1, 2)).cpu().size()=torch.Size([2, 1, 14592])
vocoder(y[0][::s].transpose(1, 2)).cpu().squeeze().size()=torch.Size([2, 14592])
y[1][::s]=tensor([57, 57], device='cuda:0'), y[1][::s].size()=torch.Size([2])
y[1][::s].squeeze()=tensor([57, 57], device='cuda:0'), y[1][::s].squeeze().size()=torch.Size([2])
DEBUG generate_audio(), type(audios)=<class 'numpy.ndarray'> type(mel_lens)=<class 'numpy.ndarray'> type(fnames)=<class 'list'>
DEBUG generate_audio(), audios.shape=(2, 14592) mel_lens.shape=(2,) len(fnames)=2

In [ ]:
if not args.skip_before_train_loop_validation:
    run_val(epoch=0)

## train loop

In [ ]:
for epoch in range(start_epoch, args.epochs + 1):
    # logging metrics
    epoch_start_time = time.perf_counter()
    iter_loss = 0
    epoch_loss = 0.0
    epoch_iter = 0
    num_iters = len(train_loader)
    mean_sl_penalty_coefs = []
    # epoch_mel_loss = 0.0
    # epoch_num_frames = 0
    # epoch_frames_per_sec = 0.0
    # iter_num_frames = 0
    # iter_meta = {}

    # iterate over all batches in epoch
    for batch in train_loader:
        if args.max_iters_per_epoch:
            if epoch_iter > args.max_iters_per_epoch:
                print("quit training loop, FOR DEVELOPMENT!!!")
                break 
            print(f'DEBUG mode iter {epoch_iter} of {args.max_iters_per_epoch}')
        
        if epoch_iter == num_iters: # useful for gradient accumulation
            break
                    
        total_iter += 1
        epoch_iter += 1
        iter_start_time = time.perf_counter()

        adjust_learning_rate(total_iter, optimizer, args.learning_rate,
                             args.warmup_steps)

        optimizer.zero_grad()

        x, y = batch_to_gpu(batch) # x: inputs, y: targets
        gt_mel = y["mel_padded"]
        
        # # y: targets
        # y = {
        #     'mel_padded': mel_padded, 
        #     'mel_lengths': mel_lengths,
        # }
        
        # forward pass through models (respeller -> quantiser -> tts)
        pred_mel, dec_lens, _g_embedding_indices = forward_pass(respeller, tts, x)
        
        # TODO: DO WE NEED MASK IF WE USE SOFTDTW LOSS? 
        # I THINK IT AUTOMATICALLY WILL ALIGN PADDED FRAMES WITH EACH OTHER???
        
        # print(f'inputs to loss {pred_mel.size()}, {gt_mel.size()}')
        
        # calculate loss
        loss = criterion(pred_mel, gt_mel)
        # print('raw loss from softdtw', loss.size())
        
        if args.avg_loss_by_speech_lens:
            loss = loss / dec_lens # needed because softdtw code doesn't return avg loss by default TODO check this!
            # TODO also add gt lens? maybe shud normalise according to path len?
            # print('loss avg according to dec seqlens', loss.size())
        
        # penalise length mismatch            
        coef = calc_sl_penalty(dec_lens, y['mel_lengths'])
        if args.speech_length_penalty_training:
            loss_no_sl_penalty = loss.clone().detach()
            loss = coef * loss
        
        loss = loss.mean()
        # print('loss avged across batch', loss.size())
        
        # backpropagation of loss
        loss.backward()
        
        # clip gradients and run optimizer
        torch.nn.utils.clip_grad_norm_(respeller.trainable_parameters(), args.grad_clip_thresh)
        optimizer.step()
        
        # log metrics to terminal and to wandb
        iter_loss = loss.item()
        iter_time = time.perf_counter() - iter_start_time
        epoch_loss += iter_loss
        
        # values to be logged by WANDB
        iter_logs = {}
        
        iter_logs["train/iter_loss"] = iter_loss
        iter_logs["train/iter_time"] = iter_time
        
        mean_sl_penalty_coef = coef.mean().item()
        mean_sl_penalty_coefs.append(mean_sl_penalty_coef)
        iter_logs["train/iter_sl_penalty_coef"] = mean_sl_penalty_coef
        
        if args.speech_length_penalty_training:
            iter_logs["train/iter_loss_no_sl_penalty"] = loss_no_sl_penalty.mean().item()
        
        if True:
            # log memory usage
            t = torch.cuda.get_device_properties(0).total_memory
            r = torch.cuda.memory_reserved(0)
            a = torch.cuda.memory_allocated(0)
            f = r-a  # free inside reserved
            iter_logs['memory/total'] = byte_to_gigabyte(t)
            iter_logs['memory/reserved'] = byte_to_gigabyte(r)
            iter_logs['memory/allocated'] = byte_to_gigabyte(a)
            iter_logs['memory/free'] = byte_to_gigabyte(f)
        
        wandb.log(iter_logs)
        

        ### Finished Epoch!
             
    epoch_time = time.perf_counter() - epoch_start_time
    
    epoch_logs = {
        "train/epoch_num": epoch,
        "train/epoch_time": epoch_time,
        "train/epoch_loss": epoch_loss / epoch_iter,
    }
    if args.speech_length_penalty_training:
        epoch_logs["train/epoch_sl_penalty_coef"] = sum(mean_sl_penalty_coefs) / len(mean_sl_penalty_coefs)
    
    wandb.log(epoch_logs)
    
    run_val(epoch=epoch)

    maybe_save_checkpoint(args, respeller, optimizer, 
                          epoch, total_iter, model_config)

    logger.flush()
        
print("\n *** Finished training! ***")

# wandb.finish() # useful in jupyter notebooks

# DEBUG SOFTDTW len normalisation issue!

In [ ]:
criterion = SoftDTW(use_cuda=True, gamma=0.1)

# test softdtw func to see if it avgs for different len n or m
device = y['mel_padded'].device

for X_len in range(1,10):
    for Y_len in range(1,10):

        X = torch.zeros(32, X_len,1).to(device)
        Y = torch.ones(32, Y_len,1).to(device)

        softdtw_loss = criterion(X, Y)
        
        normed_loss = sum(softdtw_loss) / (X_len + Y_len)
        
        print(f"{X_len=}, {Y_len=}, {sum(softdtw_loss).item()=}, {normed_loss.item()=}")

In [ ]:
criterion = SoftDTW(use_cuda=True, gamma=0.1)

# test softdtw func to see if it avgs for different len n or m
device = y['mel_padded'].device

for X_num_ones in range(1,10):
    X = torch.cat([torch.ones(32, X_num_ones, 1), torch.zeros(32, 5, 1)], dim=1).to(device)
    Y = torch.cat([torch.ones(32, 5, 1), torch.zeros(32, 5, 1)], dim=1).to(device)

    softdtw_loss = criterion(X, Y)

    normed_loss = sum(softdtw_loss) / (X_len + Y_len)

    print(f"{X_num_ones=}, {sum(softdtw_loss).item()=}, {normed_loss.item()}")

In [ ]:
criterion = SoftDTW(use_cuda=True, gamma=0.01)

# test softdtw func to see if it avgs for different len n or m
device = y['mel_padded'].device

for X_num_ones in range(1,10):
    X = torch.cat([torch.ones(32, X_num_ones, 1), torch.zeros(32, 5, 1)], dim=1).to(device)
    Y = torch.cat([torch.ones(32, 5, 1), torch.zeros(32, 5, 1)], dim=1).to(device)

    softdtw_loss = criterion(X, Y)

    normed_loss = sum(softdtw_loss) / (X_len + Y_len)

    print(f"{X_num_ones=}, {sum(softdtw_loss).item()=}, {normed_loss.item()}")

In [ ]:
criterion = SoftDTW(use_cuda=True, gamma=0.01, bandwidth=120)

# test softdtw func to see if it avgs for different len n or m
device = y['mel_padded'].device

for X_num_ones in range(1,10):
    X = torch.cat([torch.ones(32, X_num_ones, 1), torch.zeros(32, 5, 1)], dim=1).to(device)
    Y = torch.cat([torch.ones(32, 5, 1), torch.zeros(32, 5, 1)], dim=1).to(device)

    softdtw_loss = criterion(X, Y)

    normed_loss = sum(softdtw_loss) / (X_len + Y_len)

    print(f"{X_num_ones=}, {sum(softdtw_loss).item()=}, {normed_loss.item()}")

# DEBUG L1 loss

In [ ]:
def mean_absolute_error(x, y):
    """for calculating softdtw using L1 loss
    Calculates the Euclidean distance between each element in x and y per timestep
    """
    n = x.size(1)
    m = y.size(1)
    d = x.size(2)
    x = x.unsqueeze(2).expand(-1, n, m, d)
    y = y.unsqueeze(1).expand(-1, n, m, d)
    return torch.abs(x - y).sum(3)

criterion = SoftDTW(use_cuda=True, gamma=0.01, bandwidth=120, dist_func=mean_absolute_error)

# test softdtw func to see if it avgs for different len n or m
device = y['mel_padded'].device

for X_num_ones in range(1,10):
    X = torch.cat([torch.ones(32, X_num_ones, 1), torch.zeros(32, 5, 1)], dim=1).to(device)
    Y = torch.cat([torch.ones(32, 5, 1), torch.zeros(32, 5, 1)], dim=1).to(device)

    softdtw_loss = criterion(X, Y)

    normed_loss = sum(softdtw_loss) / (X_len + Y_len)

    print(f"{X_num_ones=}, {sum(softdtw_loss).item()=}, {normed_loss.item()}")